In [4]:
import runpod
from runpod.serverless.utils import rp_download

In [ ]:
## S3 Credentials
access_key = 'user_2uWqhla5LBINeGm5dUvZYUEn8S5'
secret_access_key = 'rps_17LB5XU7CLYM2OZZPSFKUHVWPCTJMR97AGAYVUIF1347qn'
bucket_name = '7v9hhny2yu'
endpoint_url = 'https://s3api-eu-cz-1.runpod.io'

In [ ]:
# 업로드/다운로드 테스트 실행 (boto3 필요)
import boto3, uuid, io


s3 = boto3.client('s3', endpoint_url=endpoint_url,
                  aws_access_key_id=access_key,
                  aws_secret_access_key=secret_access_key)

# 테스트 바이트 데이터 생성
data = (b'runpod-s3-test:' + uuid.uuid4().hex.encode())
key = f'test/test_upload_{uuid.uuid4().hex}.bin'

# 업로드
print('Uploading ->', key)
s3.put_object(Bucket=bucket_name, Key=key, Body=data)
print('Upload complete')

# 다운로드 및 검증
resp = s3.get_object(Bucket=bucket_name, Key=key)
downloaded = resp['Body'].read()
print('Downloaded bytes length:', len(downloaded))
if downloaded == data:
    print('SUCCESS: 업로드/다운로드 일치')
else:
    print('FAIL: 데이터 불일치')

# presigned URL 예시 (GET) — 외부에 전달해서 다운로드 허용할 때 사용
get_url = s3.generate_presigned_url('get_object', Params={'Bucket': bucket_name, 'Key': key}, ExpiresIn=3600)
print('Presigned GET URL (valid 1h):')
print(get_url)

# (선택) 테스트 파일 삭제하려면 아래 주석 해제
# s3.delete_object(Bucket=bucket_name, Key=key)
# print('Deleted test object.')


Uploading -> test/test_upload_720437adaa8948c0a51b9346b3fb4c77.bin


ClientError: An error occurred (AccessDenied) when calling the PutObject operation: the authorization header is malformed; the region 'us-east-1' is wrong; expecting 'eu-cz-1'

In [ ]:
import os, uuid, requests, json, boto3

S3_ENDPOINT   = os.getenv("S3_ENDPOINT")      # AWS S3 또는 RunPod S3-compatible endpoint
S3_BUCKET     = os.getenv("S3_BUCKET")
RUNPOD_ID     = os.getenv("RUNPOD_ENDPOINT_ID")
RUNPOD_APIKEY = os.getenv("RUNPOD_API_KEY")

def get_presigned_urls(filename: str):
    s3 = boto3.client("s3", endpoint_url=S3_ENDPOINT)
    key = f"uploads/{uuid.uuid4()}-{filename}"
    put_url = s3.generate_presigned_url("put_object",
        Params={"Bucket": S3_BUCKET, "Key": key}, ExpiresIn=600)
    get_url = s3.generate_presigned_url("get_object",
        Params={"Bucket": S3_BUCKET, "Key": key}, ExpiresIn=3600)
    return put_url, get_url

def call_runpod(file_get_url: str):
    payload = {"input": {"file_url": file_get_url}}
    r = requests.post(
        f"https://api.runpod.ai/v2/{RUNPOD_ID}/run",
        headers={"Authorization": f"Bearer {RUNPOD_APIKEY}",
                 "Content-Type": "application/json"},
        data=json.dumps(payload),
        timeout=30
    )
    return r.json()

# 1) 클라이언트가 put_url로 업로드 → 2) 우리 서버는 get_url을 워커에 전달
put_url, get_url = get_presigned_urls("sample.bin")
# (클라이언트는 put_url로 HTTP PUT 업로드)
job = call_runpod(get_url)
print(job)
